# Experiment on AAI Model Architectures
- Created: 10 Mar 2020
- Data: param_draft1 40k
- Increase data size and change model architecture
- Identity SE, between (cnn and bilstm) embedding feature 
- expeirment 20 21

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from os.path import join
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks
from time import time
from datetime import datetime
import shutil 
import argparse
from functools import partial
import pickle
from IPython.display import Audio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Reshape, GRU, InputLayer, AlphaDropout, Activation, BatchNormalization, Dropout, Flatten, Dense, Bidirectional, LSTM, Conv1D, SpatialDropout1D, Concatenate, Multiply, Add

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder

In [2]:
np_load_old = partial(np.load)
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

In [3]:
tf.random.set_seed(42)

## Metric Function

In [4]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred), axis = 0) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true, axis = 0)), axis = 0) 
    return K.mean(1 - (SS_res/SS_tot), axis=0)

## Utility Functions

In [5]:
#Function for preprocessing data
def delete_params(params):
    '''
    This function remove JX, WC, TRX, TRY, and MS1,2,3 paramter
    '''
    DEL_PARAMS_LIST = [2,8,15,16,21,22,23]
    return np.delete(params,DEL_PARAMS_LIST , axis=1)

In [6]:
def compute_rmse(actual,pred, axis=1):
    # Compute RMSE by row (axis=1) result in rmse of each data
    # Compute RMSE by column (axis=0) result in rmse of each label
    return np.sqrt((np.square(actual - pred)).mean(axis=axis))

In [7]:
def evaluate_model(y_pred, label):
    y_pred = enc.inverse_transform(y_pred)
    label = enc.inverse_transform(label)
#     print('Confusion Matrix')
#     print(confusion_matrix(label, y_pred))
    print('Classification Report')
    print(classification_report(label, y_pred))

In [8]:
def compute_se(actual,pred):
    return np.sqrt((np.square(actual - pred)))

## Load Data 

In [9]:
def prep_data():

    # load data from preprocess pipeline
    dataset = np.load(join('../../data/d_dataset_p1/prep_data_13_2','training_subsets.npz'))
    X_train =dataset['X_train']
    y_train= dataset['y_train']
    z_train= dataset['z_train']
    X_val = dataset['X_val']
    y_val = dataset['y_val']
    z_val = dataset['z_val']
    X_test = dataset['X_test']
    y_test = dataset['y_test']
    z_test = dataset['z_test']
    a_test = dataset['audio_test']
    a_val = dataset['audio_val']
    
    y_train = delete_params(y_train)
    y_val = delete_params(y_val)
    y_test = delete_params(y_test)

    print('Train features and labels %s %s'%(str(X_train.shape),str(y_train.shape)))
    print('Validating features and labels %s %s'%(str(X_val.shape),str(y_val.shape)))
    print('Test features and labels %s %s'%(str(X_test.shape),str(y_test.shape)))

    return X_train, X_val, X_test, y_train, y_val, y_test, z_train, z_val, z_test, a_test, a_val

In [10]:
X_train, X_val, X_test, y_train, y_val, y_test, z_train, z_val, z_test, a_test, a_val = prep_data()

Train features and labels (98746, 24, 39) (98746, 17)
Validating features and labels (12094, 24, 39) (12094, 17)
Test features and labels (12092, 24, 39) (12092, 17)


In [11]:
print(set(z_test))

{'i', 'E', 'V', '&', 'U', '9', 'u', 'O', 'e', 'M', '2', 'E:', '7', 'y', 'a', '@', 'o', 'Q', 'A'}


In [12]:
print('Train features and labels {} {} {}'.format(str(X_train.shape),str(y_train.shape),str(z_train.shape)))
print('Validating features and labels {} {} {}'.format(str(X_val.shape),str(y_val.shape),str(z_val.shape)))
print('Test features and labels {} {} {}'.format(str(X_test.shape),str(y_test.shape),str(z_test.shape)))

Train features and labels (98746, 24, 39) (98746, 17) (98746,)
Validating features and labels (12094, 24, 39) (12094, 17) (12094,)
Test features and labels (12092, 24, 39) (12092, 17) (12092,)


### One-hot label 

In [13]:
z_train_oe=z_train.reshape(-1,1)
z_val_oe=z_val.reshape(-1,1)
z_test_oe=z_test.reshape(-1,1)

enc = OneHotEncoder(handle_unknown = 'ignore')
enc.fit(z_train_oe)

z_train_oe = enc.transform(z_train_oe).toarray()
z_test_oe = enc.transform(z_test_oe).toarray()
z_val_oe = enc.transform(z_val_oe).toarray()

## 1.) Train Baseline Model
- Model without embedding feature

### Defined Model without Embedded Feature

In [14]:
N_OUTPUTS = 17

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)
pConv1D = partial(Conv1D, padding = 'same', activation = 'linear', kernel_initializer = 'he_uniform')

def init_senet(feature_layer=1, cnn_unit=128, first_kernel=13, res_unit=128, res_kernel=7,
    bilstm = 1, bilstm_unit=256, 
    dropout_rate=0.3,
    reduction_ratio = 2,
    activation_fn='relu'):


    def cnn_block(input_x, cnn_unit, kernel_size):
        x = pConv1D(cnn_unit, kernel_size=kernel_size)(input_x)
        x = BatchNormalization()(x)
        x = Activation(activation_fn)(x)
        return x
    
    def residual_block(input_x):
        x = pConv1D(res_unit, kernel_size=res_kernel)(input_x)
        x = BatchNormalization()(x)
        x = Activation(activation_fn)(x)
        x = pConv1D(res_unit, kernel_size=3)(x)
        return x

    def se_block(input_x):
        x = layers.GlobalAveragePooling1D()(input_x)
        channel_shape = getattr(x, '_shape_val')[-1]
        x = Reshape((1, channel_shape))(x)
        x = Dense(channel_shape // reduction_ratio, activation=activation_fn, kernel_initializer='he_uniform')(x)
        x = Dense(channel_shape, activation='tanh', kernel_initializer='he_uniform')(x)
        x = layers.Multiply()([x, input_x])
        return x

    def se_res_block(input_x):
        se_x = se_block(input_x)
        re_x = residual_block(input_x)
        x = keras.layers.Add()([re_x, se_x])
        x = BatchNormalization()(x)
        output = Activation(activation_fn)(x)
        return x

    def senet_nn(input_shape_1,input_shape_2):

        input_x = keras.Input(shape=(input_shape_1,input_shape_2))
        x = cnn_block(input_x, cnn_unit, first_kernel)
        for i in range(feature_layer):
            x = se_res_block(x)
        x = SpatialDropout1D(rate=dropout_rate)(x)
        for i in range(feature_layer):
            x = se_res_block(x)
            x = SpatialDropout1D(rate=dropout_rate)(x)
#         x = cnn_block(x, cnn_unit=cnn_unit, kernel_size=1)
#         x = SpatialDropout1D(rate=dropout_rate)(x)
        for i in range(bilstm-1):
            x = Bidirectional(pLSTM(bilstm_unit))(x)
            x = SpatialDropout1D(rate=dropout_rate)(x)
        x = Bidirectional(pLSTM(bilstm_unit, return_sequences=False))(x)
        x = Dropout(rate=dropout_rate)(x)
        outputs = Dense(N_OUTPUTS, activation='linear', kernel_initializer='he_uniform')(x)
        model = keras.Model(inputs=input_x, outputs=outputs)
        return model

    return senet_nn

In [15]:
model = init_senet()(24, 39)

In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 24, 39)]     0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 24, 128)      65024       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 24, 128)      512         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 24, 128)      0           batch_normalization[0][0]        
______________________________________________________________________________________________

### Compile Model 

In [17]:
model.compile(optimizer='adam',loss='mse',metrics=[rmse, R2])

### Train Model 

In [18]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 100

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [19]:
history = model.fit(X_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val,y_val),
    callbacks=callback_list)

Train on 98746 samples, validate on 12094 samples
Epoch 1/100
98746/98746 [==============================] - 28s 285us/sample - loss: 0.0929 - rmse: 0.2808 - R2: 0.5937 - val_loss: 0.0449 - val_rmse: 0.1893 - val_R2: 0.8063
Epoch 2/100
98746/98746 [==============================] - 19s 188us/sample - loss: 0.0598 - rmse: 0.2222 - R2: 0.7427 - val_loss: 0.0433 - val_rmse: 0.1843 - val_R2: 0.8143
Epoch 3/100
98746/98746 [==============================] - 20s 206us/sample - loss: 0.0538 - rmse: 0.2093 - R2: 0.7682 - val_loss: 0.0413 - val_rmse: 0.1788 - val_R2: 0.8221
Epoch 4/100
98746/98746 [==============================] - 20s 202us/sample - loss: 0.0501 - rmse: 0.2011 - R2: 0.7841 - val_loss: 0.0441 - val_rmse: 0.1817 - val_R2: 0.8084
Epoch 5/100
98746/98746 [==============================] - 20s 200us/sample - loss: 0.0477 - rmse: 0.1959 - R2: 0.7946 - val_loss: 0.0430 - val_rmse: 0.1785 - val_R2: 0.8171
Epoch 6/100
98746/98746 [==============================] - 20s 201us/sample - lo

In [20]:
model.save('model_baseline_dp1.hdf5')

### Evaluation 

In [21]:
model.evaluate(X_test,y_test,verbose=0)

[0.03393776446033739, 0.15680017, 0.8459989]

In [22]:
y_pred = model.predict(X_test)

In [23]:
err = compute_rmse(y_test,y_pred, axis=0)

In [24]:
print(err)

[0.16298977 0.1422034  0.15376249 0.20005407 0.16684956 0.22112424
 0.25571529 0.10210641 0.13830007 0.24698465 0.14288077 0.16797964
 0.17510188 0.27332954 0.18182119 0.15415196 0.15161589]


## 2.) Model with (Between) Embedding Feature  

### Defined (Between) Embedding Model

In [25]:
PHONETIC_OUTPUT = len(set(z_train))
EMBEDDED_FEATURE = 128

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)

def embeded_model(input_shape_1,input_shape_2):
    inputs = keras.Input(shape=(input_shape_1,input_shape_2))
    x = pLSTM(EMBEDDED_FEATURE)(inputs)
    x = Flatten()(x)
    outputs = Dense(PHONETIC_OUTPUT, activation='softmax', kernel_initializer='he_uniform')(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [26]:
embedded = embeded_model(24, 39)
embedded.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 24, 39)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 24, 128)           86016     
_________________________________________________________________
flatten (Flatten)            (None, 3072)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 19)                58387     
Total params: 144,403
Trainable params: 144,403
Non-trainable params: 0
_________________________________________________________________


### Compile Model 

In [27]:
embedded.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

### Train Model 

In [28]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 100

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [29]:
history = embedded.fit(X_train,z_train_oe,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(X_val,z_val_oe),
    callbacks=callback_list)

Train on 98746 samples, validate on 12094 samples
Epoch 1/100
98746/98746 - 6s - loss: 2.2888 - accuracy: 0.7590 - val_loss: 2.1787 - val_accuracy: 0.8577
Epoch 2/100
98746/98746 - 4s - loss: 2.1934 - accuracy: 0.8429 - val_loss: 2.1607 - val_accuracy: 0.8739
Epoch 3/100
98746/98746 - 4s - loss: 2.1671 - accuracy: 0.8681 - val_loss: 2.1500 - val_accuracy: 0.8844
Epoch 4/100
98746/98746 - 4s - loss: 2.1511 - accuracy: 0.8838 - val_loss: 2.1510 - val_accuracy: 0.8823
Epoch 5/100
98746/98746 - 4s - loss: 2.1400 - accuracy: 0.8942 - val_loss: 2.1447 - val_accuracy: 0.8873
Epoch 6/100
98746/98746 - 4s - loss: 2.1325 - accuracy: 0.9013 - val_loss: 2.1374 - val_accuracy: 0.8952
Epoch 7/100
98746/98746 - 4s - loss: 2.1252 - accuracy: 0.9081 - val_loss: 2.1369 - val_accuracy: 0.8955
Epoch 8/100
98746/98746 - 4s - loss: 2.1204 - accuracy: 0.9128 - val_loss: 2.1389 - val_accuracy: 0.8929
Epoch 9/100
98746/98746 - 4s - loss: 2.1149 - accuracy: 0.9184 - val_loss: 2.1391 - val_accuracy: 0.8936
Epoch

In [30]:
embedded.save('between_embedded_dp1.hdf5')

In [31]:
z_pred = embedded.predict(X_test)
evaluate_model(z_pred, z_test_oe)

Classification Report
              precision    recall  f1-score   support

           &       0.94      0.89      0.92       655
           2       0.94      0.97      0.95       636
           7       0.94      0.55      0.70       552
           9       0.93      0.91      0.92       625
           @       0.99      0.87      0.92       647
           A       0.85      0.95      0.90       619
           E       0.84      0.96      0.90       652
          E:       0.97      0.93      0.95       629
           M       0.84      0.94      0.89       650
           O       0.71      0.91      0.80       674
           Q       0.94      0.86      0.90       670
           U       0.90      0.77      0.83       641
           V       0.70      0.78      0.74       644
           a       0.93      0.90      0.91       644
           e       0.92      0.96      0.94       661
           i       0.98      0.90      0.94       650
           o       0.94      0.89      0.91       571
     

### Embedded input model

In [32]:
# embedded = tf.keras.models.load_model('pre_embedded.hdf5')

embedded_layers = tf.keras.Sequential()
for layer in embedded.layers[:-2]:
    layer.trainable = False
    embedded_layers.add(layer)
embedded_layers.summary()

N_OUTPUTS = 17

pLSTM = partial(LSTM, kernel_initializer='he_uniform', return_sequences=True)
pConv1D = partial(Conv1D, padding = 'same', activation = 'linear', kernel_initializer = 'he_uniform')

def init_senet_w_emb(feature_layer=1, cnn_unit=128, first_kernel=13, res_unit=128, res_kernel=7,
    bilstm = 1, bilstm_unit=256, 
    dropout_rate=0.3,
    reduction_ratio = 2,
    activation_fn='relu'):

    def cnn_block(input_x, cnn_unit, kernel_size):
        x = pConv1D(cnn_unit, kernel_size=kernel_size)(input_x)
        x = BatchNormalization()(x)
        x = Activation(activation_fn)(x)
        return x
    
    def residual_block(input_x):
        x = pConv1D(res_unit, kernel_size=res_kernel)(input_x)
        x = BatchNormalization()(x)
        x = Activation(activation_fn)(x)
        x = pConv1D(res_unit, kernel_size=3)(x)
        return x

    def se_block(input_x):
        x = layers.GlobalAveragePooling1D()(input_x)
        channel_shape = getattr(x, '_shape_val')[-1]
        x = Reshape((1, channel_shape))(x)
        x = Dense(channel_shape // reduction_ratio, activation=activation_fn, kernel_initializer='he_uniform')(x)
        x = Dense(channel_shape, activation='tanh', kernel_initializer='he_uniform')(x)
        x = layers.Multiply()([x, input_x])
        return x

    def se_res_block(input_x):
        se_x = se_block(input_x)
        re_x = residual_block(input_x)
        x = keras.layers.Add()([re_x, se_x])
        x = BatchNormalization()(x)
        output = Activation(activation_fn)(x)
        return x

    def senet_nn(input_shape_1,input_shape_2):

        input_x = keras.Input(shape=(input_shape_1,input_shape_2))
        x = cnn_block(input_x, cnn_unit, first_kernel)
        for i in range(feature_layer):
            x = se_res_block(x)
        x = SpatialDropout1D(rate=dropout_rate)(x)
        for i in range(feature_layer):
            x = se_res_block(x)
            x = SpatialDropout1D(rate=dropout_rate)(x)
#         x = cnn_block(x, cnn_unit=cnn_unit, kernel_size=1)
        # Embedded
        embedded = embedded_layers(input_x)
        x = layers.Concatenate()([x, embedded])
#         x = SpatialDropout1D(rate=dropout_rate)(x)
        for i in range(bilstm-1):
            x = Bidirectional(pLSTM(bilstm_unit))(x)
            x = SpatialDropout1D(rate=dropout_rate)(x)
        x = Bidirectional(pLSTM(bilstm_unit, return_sequences=False))(x)
        x = Dropout(rate=dropout_rate)(x)
        outputs = Dense(N_OUTPUTS, activation='linear', kernel_initializer='he_uniform')(x)
        model = keras.Model(inputs=input_x, outputs=outputs)
        return model

    return senet_nn

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 24, 128)           86016     
Total params: 86,016
Trainable params: 0
Non-trainable params: 86,016
_________________________________________________________________


In [33]:
emb_model = init_senet_w_emb()(24, 39)
emb_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 24, 39)]     0                                            
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 24, 128)      65024       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_5 (BatchNor (None, 24, 128)      512         conv1d_5[0][0]                   
__________________________________________________________________________________________________
activation_5 (Activation)       (None, 24, 128)      0           batch_normalization_5[0][0]      
____________________________________________________________________________________________

### Compile Model 

In [34]:
emb_model.compile(optimizer='adam',loss='mse',metrics=[rmse, R2])

### Train Model 

In [35]:
PATIENCE = 10
BATCH_SIZE = 128
EPOCHS = 100

early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=PATIENCE, verbose=1, mode='min', baseline=None, restore_best_weights=False)
callback_list = [early]

In [36]:
history = emb_model.fit(X_train,y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    verbose=2,
    validation_data=(X_val,y_val),
    callbacks=callback_list)

Train on 98746 samples, validate on 12094 samples
Epoch 1/100
98746/98746 - 30s - loss: 0.0605 - rmse: 0.2230 - R2: 0.7344 - val_loss: 0.0374 - val_rmse: 0.1713 - val_R2: 0.8398
Epoch 2/100
98746/98746 - 21s - loss: 0.0364 - rmse: 0.1745 - R2: 0.8434 - val_loss: 0.0363 - val_rmse: 0.1639 - val_R2: 0.8444
Epoch 3/100
98746/98746 - 21s - loss: 0.0333 - rmse: 0.1665 - R2: 0.8566 - val_loss: 0.0364 - val_rmse: 0.1624 - val_R2: 0.8450
Epoch 4/100
98746/98746 - 21s - loss: 0.0316 - rmse: 0.1620 - R2: 0.8636 - val_loss: 0.0353 - val_rmse: 0.1606 - val_R2: 0.8483
Epoch 5/100
98746/98746 - 21s - loss: 0.0304 - rmse: 0.1588 - R2: 0.8687 - val_loss: 0.0353 - val_rmse: 0.1599 - val_R2: 0.8483
Epoch 6/100
98746/98746 - 21s - loss: 0.0290 - rmse: 0.1555 - R2: 0.8742 - val_loss: 0.0358 - val_rmse: 0.1587 - val_R2: 0.8468
Epoch 7/100
98746/98746 - 21s - loss: 0.0279 - rmse: 0.1525 - R2: 0.8789 - val_loss: 0.0351 - val_rmse: 0.1582 - val_R2: 0.8491
Epoch 8/100
98746/98746 - 21s - loss: 0.0269 - rmse: 0

In [37]:
emb_model.save('model_with_between_embedded_dp1.hdf5')

## Evaluated 

In [38]:
emb_model.evaluate(X_test,y_test,verbose=0)

[0.03432742265594388, 0.1549069, 0.8438866]

In [39]:
y_pred = emb_model.predict(X_test)

In [40]:
compute_rmse(y_test,y_pred, axis=0)

array([0.16488171, 0.14267572, 0.15189093, 0.20126693, 0.1664796 ,
       0.219056  , 0.26207463, 0.10280186, 0.13552491, 0.24797773,
       0.14020805, 0.16877097, 0.17340989, 0.27765885, 0.1818688 ,
       0.16000507, 0.15377611])